# Compute Skill Metrics for certain lead week
## Test data is unbalanced


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: marcodia
"""
import numpy as np
import random
import xarray as xr
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import math

from sklearn import preprocessing
import tensorflow as tf

import import_ipynb
import sys
import os 

import network_arch as network
import metrics
import plot as plot
import settings
import functions_misc as fnc


from cartopy import config
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry.polygon import LinearRing

import pop_tools


2023-10-12 09:34:48.162890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from network_arch.ipynb
importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from plot.ipynb
importing Jupyter notebook from settings.ipynb
importing Jupyter notebook from functions_misc.ipynb


# Functions

In [2]:
# # MAKE THE NN ARCHITECTURE
def make_model():
    # Define and train the model
    tf.keras.backend.clear_session()
    model = network.defineNN(HIDDENS,
                             input1_shape = X_train.shape[1],
                             output_shape=NLABEL,
                             ridge_penalty1=RIDGE1,
                             dropout=DROPOUT,
                             act_fun='relu',
                             network_seed=NETWORK_SEED)
    
    loss_function = tf.keras.losses.CategoricalCrossentropy()    
    model.compile(
                  optimizer = tf.keras.optimizers.Adam(learning_rate=LR_INIT),
                  loss = loss_function,
                  metrics = [
                      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
                      metrics.PredictionAccuracy(NLABEL)
                      ]
                  )           
    return model, loss_function

In [3]:
def is_mam(month):
    return np.logical_and(month>=3, month<=5)

def is_may(month):
    return np.logical_and(month>=5, month<=5)

def is_aug(month):
    return np.logical_and(month>=8, month<=8)

In [4]:
#.............................................
# XAI functions
#.............................................

# before calling these functions in your notebook, make sure you have defined the tensorflow object "model". 
# The "model" is the machine learning model (e.g., neural network) that you want to explain. 

# Code author of XAI functions: Libby Barnes and Tony Mamalakis

def get_gradients(inputs, top_pred_idx=None):
    """Computes the gradients of outputs w.r.t input image.

    Args:
        inputs: 2D/3D/4D matrix of samples
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.

    Returns:
        Gradients of the predictions w.r.t img_input
    """
    inputs = tf.cast(inputs, tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        
        # Run the forward pass of the layer and record operations
        # on GradientTape.
        preds = model(inputs, training=False)  
        
        # For classification, grab the top class
        if top_pred_idx is not None:
            preds = preds[:, top_pred_idx]
        
    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.        
    grads = tape.gradient(preds, inputs)
    return grads

def get_integrated_gradients(inputs, baseline=None, num_steps=50, top_pred_idx=None):
    """Computes Integrated Gradients for a prediction.

    Args:
        inputs (ndarray): 2D/3D/4D matrix of samples
        baseline (ndarray): The baseline image to start with for interpolation
        num_steps: Number of interpolation steps between the baseline
            and the input used in the computation of integrated gradients. These
            steps along determine the integral approximation error. By default,
            num_steps is set to 50.
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.            

    Returns:
        Integrated gradients w.r.t input image
    """
    # If baseline is not provided, start with zeros
    # having same size as the input image.
    if baseline is None:
        input_size = np.shape(inputs)[1:]
        baseline = np.zeros(input_size).astype(np.float32)
    else:
        baseline = baseline.astype(np.float32)

    # 1. Do interpolation.
    inputs = inputs.astype(np.float32)
    interpolated_inputs = [
        baseline + (step / num_steps) * (inputs - baseline)
        for step in range(num_steps + 1)
    ]
    interpolated_inputs = np.array(interpolated_inputs).astype(np.float32)

    # 3. Get the gradients
    grads = []
    for i, x_data in enumerate(interpolated_inputs):
        grad = get_gradients(x_data, top_pred_idx=top_pred_idx)
        grads.append(grad)
    grads = tf.convert_to_tensor(grads, dtype=tf.float32)

    # 4. Approximate the integral using the trapezoidal rule
    grads = (grads[:-1] + grads[1:]) / 2.0
    avg_grads = tf.reduce_mean(grads, axis=0)

    # 5. Calculate integrated gradients and return
    integrated_grads = (inputs - baseline) * avg_grads
    return integrated_grads

def random_baseline_integrated_gradients(inputs, num_steps=50, num_runs=5, top_pred_idx=None):
    """Generates a number of random baseline images.

    Args:
        inputs (ndarray): 2D/3D/4D matrix of samples
        num_steps: Number of interpolation steps between the baseline
            and the input used in the computation of integrated gradients. These
            steps along determine the integral approximation error. By default,
            num_steps is set to 50.
        num_runs: number of baseline images to generate
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.      

    Returns:
        Averaged integrated gradients for `num_runs` baseline images
    """
    # 1. List to keep track of Integrated Gradients (IG) for all the images
    integrated_grads = []

    # 2. Get the integrated gradients for all the baselines
    for run in range(num_runs):
        baseline = np.zeros(np.shape(inputs)[1:])
        for i in np.arange(0,np.shape(baseline)[0]):
            j = np.random.choice(np.arange(0,np.shape(inputs)[0]))
            baseline[i] = inputs[j,i]

        igrads = get_integrated_gradients(
            inputs=inputs,
            baseline=baseline,
            num_steps=num_steps,
        )
        integrated_grads.append(igrads)

    # 3. Return the average integrated gradients for the image
    integrated_grads = tf.convert_to_tensor(integrated_grads)
    return tf.reduce_mean(integrated_grads, axis=0)



# Code

In [235]:
min_lon = 265 
min_lat = 8 
max_lon = 320 
max_lat = 50 

rolling_input_window_X = 3
rolling_input_window_Y = 1

lambda_value = [0,7,14,21,28,35,42,49,56,63,70,77,84]
fill_val = np.arange(0,120)

poisson_weights = np.zeros((len(lambda_value),len(fill_val)))
k_lead = 119 

num_ens_test = 5
exp_ind = 0
threat_score_l8 = np.zeros(num_ens_test)
gilbert_skill_score_l8 = np.zeros(num_ens_test)

In [236]:
for exp in ['exp_8/exp_800', 'exp_8/exp_801','exp_8/exp_802','exp_8/exp_803','exp_8/exp_804']:
    EXPERIMENT = exp     

    ddir_X = '/Users/marcodia/Research/Data/global_daily_anomalies/'
    ddir_Y = '/Users/marcodia/Research/Data/processed_fields/precip_data/'
    ddir_out = '/Users/marcodia/Research/salinity_s2s/experiments/no_arctic/exp_8/' 
    ddir_MODEL = ddir_out
    ddir_saveinfo = '/Users/marcodia/Research/salinity_s2s/experiments/no_arctic/evaluations/'

    params = settings.get_settings(EXPERIMENT)

    PREDICTOR_VAR  = params['PREDICTOR_VAR']           
    PREDICTAND_VAR = params['PREDICTAND_VAR']              
    REGION_TOR     = params['REGION_TOR']          
    REGION_TAND    = params['REGION_TAND']            
    training_ens   = params['training_ens']            
    validation_ens = params['validation_ens']           
    testing_ens    = params['testing_ens']           
    train_list     = params['train_list']
    val_list       = params['val_list']
    lead           = params['lead']            
    days_average   = params['days_average']            
    GLOBAL_SEED    = params['GLOBAL_SEED']            
    HIDDENS        = params['HIDDENS']          
    DROPOUT        = params['DROPOUT']            
    RIDGE1         = params['RIDGE1']                    
    LR_INIT        = params['LR_INIT']
    BATCH_SIZE     = params['BATCH_SIZE']           
    RANDOM_SEED    = params['RANDOM_SEED']            
    act_fun        = params['act_fun']            
    N_EPOCHS       = params['N_EPOCHS']           
    PATIENCE       = params['PATIENCE']   
    CLASS_WEIGHT   = 'none'

    window_size = days_average

    lead_weeks = int(lead/7)

    #>>>>>SET UP <<<<<<<<<<<<<<<
    np.random.seed(GLOBAL_SEED)
    random.seed(GLOBAL_SEED)
    tf.compat.v1.random.set_random_seed(GLOBAL_SEED)

    NLABEL = 2

    YEARS = '1850-1949'
    STRT_I = pd.to_datetime('01-01-1850')
    END_I   = pd.to_datetime('12-31-1949')  + dt.timedelta(days=1)

    time_range = xr.cftime_range(str(STRT_I)[:10], str(END_I)[:10],calendar = 'noleap') #[0:10] corresponds to full datestamp
    #time_range_szn = time_range.where(fnc.is_mjja(time_range.month)).dropna()
    TIME_ALL = xr.DataArray(time_range + dt.timedelta(days=0), dims=['time'])     

    STRT_2 = pd.to_datetime('05-01-1850')
    END_2   = pd.to_datetime('08-31-1949')  + dt.timedelta(days=1)
    time_range2 = xr.cftime_range(str(STRT_2)[:10], str(END_2)[:10],calendar = 'noleap') #[0:10] corresponds to full datestamp
    time_range_2 = time_range2.where(fnc.is_mjja(time_range2.month)).dropna()
    TIME_X = xr.DataArray(time_range_2 + dt.timedelta(days=0), dims=['time'])   

    TIME_Y = xr.DataArray(time_range_2 + dt.timedelta(days=lead+days_average), dims=['time'])  #below comment explains time segmentation


    # ----- X TRAINING ------
    count = 0 
    for i in train_list:
        X_finame = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+i+'_dailyanom_detrend.nc'
        X_all_full = xr.open_dataarray(ddir_X+X_finame)
        X1 = X_all_full.where(X_all_full.time == TIME_ALL, drop=True)
        X = X1.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

        X_nptime = np.array(X.time)                 #for some annoying reason, it needed to be converted to numpy for creating DataArray   
        X_nplat = np.array(X.lat)
        X_nplon = np.array(X.lon)
        del X_all_full 

        if count == 0: # don't rewrite empty matrix each time 
            X_all1_FULL = xr.DataArray(np.zeros((len(train_list),X.shape[0],X.shape[1],X.shape[2]))+np.nan,
                                 dims = ['ens','time','lat','lon'],
                                 coords = [('ens',np.arange(0,len(train_list))),('time', X_nptime),('lat',X_nplat),('lon',X_nplon)])

        X_all1_FULL[count,:,:,:] = X   

        count = count+1
        del X

    X_all1_ROLL = X_all1_FULL.rolling(time=(rolling_input_window_X),center=False).mean()
    X_all1_inputtime1 = X_all1_ROLL.where(X_all1_ROLL.time == TIME_X, drop=True)
    X_all1_inputtime = X_all1_inputtime1.dropna(dim='time', how = 'all')

    Xtrain1 = X_all1_inputtime.stack(time_all=('ens','time')) # lat,lon,time*8 (8= number of training ens members) 
    Xtrain1 = Xtrain1.transpose('time_all','lat','lon') # time*8,lat,lon
    Xtrain_std1 = np.std(Xtrain1,axis=0)
    Xtrain_mean1 = np.mean(Xtrain1,axis=0)
    Xtrain_full = (Xtrain1-Xtrain_mean1)/Xtrain_std1

    del X_all1_ROLL, X_all1_inputtime1, X1, Xtrain1

    ## Apply Mask

    # Define latitude and longitude ranges that correspond to the Pacific Ocean
    pacific_lat_range = slice(0, 3)  # Adjust the latitudes as needed
    pacific_lon_range = slice(0, 5)  # Adjust the longitudes as needed

    # Create a mask for the Pacific Ocean
    mask = np.zeros(Xtrain_full.shape[1:], dtype=bool)
    mask[pacific_lat_range, pacific_lon_range] = True

    # Expand the mask to match the 'time' dimension
    mask_expanded = np.repeat(mask[None, :, :], len(Xtrain_full.time_all), axis=0)

    # Apply the mask to the dataset
    Xtrain = Xtrain_full.where(~mask_expanded, np.nan)

    # ---------- X VALIDATION----------
    count = 0 
    for i in val_list:
        X_finame = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+i+'_dailyanom_detrend.nc'
        X_all_full = xr.open_dataarray(ddir_X+X_finame)
        X1 = X_all_full.where(X_all_full.time == TIME_ALL, drop=True)
        X = X1.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

        X_nptime = np.array(X.time)                 #for some annoying reason, it needed to be converted to numpy for creating DataArray   
        X_nplat = np.array(X.lat)
        X_nplon = np.array(X.lon)
        del X_all_full 

        if count == 0: # don't rewrite empty matrix each time 
            X_val1_FULL = xr.DataArray(np.zeros((len(val_list),X.shape[0],X.shape[1],X.shape[2]))+np.nan,
                                 dims = ['ens','time','lat','lon'],
                                 coords = [('ens',np.arange(0,len(val_list))),('time', X_nptime),('lat',X_nplat),('lon',X_nplon)])

        X_val1_FULL[count,:,:,:] = X   

        count = count+1
        del X

    X_val1_ROLL = X_val1_FULL.rolling(time=(rolling_input_window_X),center=False).mean()
    X_val1_inputtime1 = X_val1_ROLL.where(X_val1_ROLL.time == TIME_X, drop=True)
    X_val1_inputtime = X_val1_inputtime1.dropna(dim='time', how = 'all')
    Xval1 = X_val1_inputtime.stack(time_all=('ens','time')) # lat,lon,time*8 (8= number of training ens members) 
    Xval1 = Xval1.transpose('time_all','lat','lon') # time*8,lat,lon

    Xval_full = (Xval1 - Xtrain_mean1)/Xtrain_std1

    # Create a mask for the Pacific Ocean
    mask = np.zeros(Xval_full.shape[1:], dtype=bool)
    mask[pacific_lat_range, pacific_lon_range] = True

    # Expand the mask to match the 'time' dimension
    mask_expanded = np.repeat(mask[None, :, :], len(Xval_full.time_all), axis=0)

    # Apply the mask to the dataset
    Xval = Xval_full.where(~mask_expanded, np.nan)

    # ---------- X TESTING----------
    X_finame1  = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+str(testing_ens)+'_dailyanom_detrend.nc'
    Xtest1_full = xr.open_dataarray(ddir_X+X_finame1)
    Xtest_1= Xtest1_full.where(Xtest1_full.time == TIME_ALL, drop=True)
    Xtest1 = Xtest_1.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

    X_test1_ROLL = Xtest1.rolling(time=(rolling_input_window_X),center=False).mean()
    X_test1_inputtime1 = X_test1_ROLL.where(X_test1_ROLL.time == TIME_X, drop=True)
    X_test1_inputtime = X_test1_inputtime1.dropna(dim='time', how = 'all')

    Xtest_full = (X_test1_inputtime - Xtrain_mean1)/Xtrain_std1

    # Create a mask for the Pacific Ocean
    mask = np.zeros(Xtest_full.shape[1:], dtype=bool)
    mask[pacific_lat_range, pacific_lon_range] = True

    # Expand the mask to match the 'time' dimension
    mask_expanded = np.repeat(mask[None, :, :], len(Xtest_full.time), axis=0)

    # Apply the mask to the dataset
    Xtest = Xtest_full.where(~mask_expanded, np.nan)

    del Xtrain_std1, X_test1_inputtime, X_test1_inputtime1, X_test1_ROLL, Xtest1, Xtest_1, Xtest1_full, Xtrain_mean1, X_val1_inputtime, X_val1_ROLL

    ## Apply Poisson Weighting


    # Calculate Poisson weights for lambda=14 and max value=30
    count = 0
    for l in lambda_value:
        poisson_weights[count,:] = fnc.calculate_poisson_weights(l, k_lead)
        count += 1

    poisson_weights_T = np.transpose(poisson_weights)

    poisson_use = poisson_weights_T[:,lead_weeks]   #THIS NEEDS TO BE SPECIFIED BASED ON LEAD TIME

    # Example usage:
    weight_vector = poisson_use  # Symmetric weights for a centered rolling average

    center_element = lead

    #%% ----- Y TRAINING--------

    count = 0
    for i in train_list:
        Ytrain_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_'+YEARS+'_ens'+str(i)+'_'+str(window_size)+'daysum.nc'

        Y_all_full = xr.open_dataarray(ddir_Y+Ytrain_finame)
        Y = Y_all_full.where(Y_all_full.time == TIME_ALL, drop=True)

        Y_nptime = np.array(Y.time)                 
        del Y_all_full 

        if count == 0: # don't rewrite empty matrix each time 
            Y_all = xr.DataArray(np.zeros((len(train_list),Y.shape[0]))+np.nan,
                                 dims = ['ens','time'],
                                 coords = [('ens',np.arange(0,len(train_list))),('time', Y_nptime)])

            poiss_weighted_rolling_avg = xr.DataArray(np.zeros((len(train_list),Y.shape[0]))+np.nan,
                                  dims = ['ens','time'],
                                coords = [('ens',np.arange(0,len(train_list))),('time', Y_nptime)])

        Y_all[count,:] = Y   

        centered_weighted_rolling_avg = fnc.uncentered_weighted_rolling_average(Y_all[count,:], weight_vector,center_element)
        poiss_weighted_rolling_avg[count,:] = centered_weighted_rolling_avg   

        count = count + 1

    Y_all1_inputtime1 = poiss_weighted_rolling_avg.where(poiss_weighted_rolling_avg.time == TIME_Y, drop=True)
    Y_all1_inputtime = Y_all1_inputtime1.dropna(dim='time', how = 'all')
    np.arange(Y_all1_inputtime.shape[0])

    YP_nptime = np.array(Y_all1_inputtime.time)
    Y_all_perc = xr.DataArray(np.zeros((len(train_list),Y_all1_inputtime.shape[1]))+np.nan,
                                 dims = ['ens','time'],
                                 coords = [('ens',np.arange(0,len(train_list))),('time', YP_nptime)])

    for i in np.arange(Y_all1_inputtime.shape[0]):
        heavy_val = np.percentile(Y_all1_inputtime[i,:], 80)
        Y_all_perc[i,:] = (Y_all1_inputtime[i,:] >= heavy_val).astype(int) #+ (Y_use >= mod_val).astype(int)

    Ytrain = Y_all_perc.stack(time_all=('ens','time'))
    # How often does our data fall into each category? This is just for the last ensemble member in training
    calcpercent = lambda cat: str((np.sum(np.array(Ytrain) == cat)/len(Ytrain)*100).astype(int))

    # Print out the sizes of each class
    # print('Frequency for each Precip Category')
    # print('Light: ' + calcpercent(0) + '%')
    # print('Heavy: ' + calcpercent(1) + '%')

    #%% ----- Y VALIDATION--------

    count = 0
    for i in val_list:
        Yval_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_'+YEARS+'_ens'+str(i)+'_'+str(window_size)+'daysum.nc'

        Y_all_full = xr.open_dataarray(ddir_Y+Yval_finame)
        Y = Y_all_full.where(Y_all_full.time == TIME_ALL, drop=True)

        Y_nptime = np.array(Y.time)                 
        del Y_all_full 

     #   $$$$$$$
        if count == 0: # don't rewrite empty matrix each time 
            Y_all = xr.DataArray(np.zeros((len(val_list),Y.shape[0]))+np.nan,
                                 dims = ['ens','time'],
                                 coords = [('ens',np.arange(0,len(val_list))),('time', Y_nptime)])

            val_poiss_weighted_rolling_avg = xr.DataArray(np.zeros((len(val_list),Y.shape[0]))+np.nan,
                                  dims = ['ens','time'],
                                coords = [('ens',np.arange(0,len(val_list))),('time', Y_nptime)])

        Y_all[count,:] = Y   

        val_centered_weighted_rolling_avg = fnc.uncentered_weighted_rolling_average(Y_all[count,:], weight_vector, center_element)
        val_poiss_weighted_rolling_avg[count,:] = val_centered_weighted_rolling_avg   

        count = count + 1

    Y_val1_inputtime1 = val_poiss_weighted_rolling_avg.where(val_poiss_weighted_rolling_avg.time == TIME_Y, drop=True)
    Y_val1_inputtime = Y_val1_inputtime1.dropna(dim='time', how = 'all')

    YP_nptime = np.array(Y_all1_inputtime.time)
    Y_val_perc = xr.DataArray(np.zeros((len(val_list),Y_all1_inputtime.shape[1]))+np.nan,
                                 dims = ['ens','time'],
                                 coords = [('ens',np.arange(0,len(val_list))),('time', YP_nptime)])

    for i in np.arange(Y_val1_inputtime.shape[0]):
        heavy_val = np.percentile(Y_val1_inputtime[i,:], 80)
        Y_val_perc[i,:] = (Y_val1_inputtime[i,:] >= heavy_val).astype(int) #+ (Y_use >= mod_val).astype(int)

    Yval = Y_val_perc.stack(time_all=('ens','time'))
    # How often does our data fall into each category? This is just for the last ensemble member in validation
    calcpercent = lambda cat: str((np.sum(np.array(Yval) == cat)/len(Yval)*100).astype(int))

    # Print out the sizes of each class
    # print('Frequency for each Precip Category')
    # print('Light: ' + calcpercent(0) + '%')
    # print('Heavy: ' + calcpercent(1) + '%')


    # ----- Y TESTING --------
    Ytest_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_'+YEARS+'_ens'+str(testing_ens)+'_'+str(window_size)+'daysum.nc'

    Y_test_full = xr.open_dataarray(ddir_Y+Ytest_finame)
    Y = Y_test_full.where(Y_test_full.time == TIME_ALL, drop=True)
    Y_nptime = np.array(Y.time)

    c_temp = xr.DataArray(np.zeros(Y.shape[0])+np.nan,
                                  dims = ['time'],
                                coords = [('time', Y_nptime)])

    c_temp[:] = fnc.uncentered_weighted_rolling_average(Y,weight_vector,center_element)

    Y_test1_inputtime1 = c_temp.where(c_temp.time == TIME_Y, drop=True)
    Y_test1_inputtime = Y_test1_inputtime1.dropna(dim='time', how = 'all')

    heavy_val = np.percentile(Y_test1_inputtime, 80)
    Ytest = (Y_test1_inputtime >= heavy_val).astype(int) 

    calcpercent = lambda cat: str((np.sum(np.array(Ytest) == cat)/len(Ytest)*100).astype(int))

    # Print out the sizes of each class
    # print('Frequency for each Precip Category')
    # print('Light: ' + calcpercent(0) + '%')
    # print('Heavy: ' + calcpercent(1) + '%')

    #Balance Classes
    #Training
    # make Yval have equal 0s and 1s so that random chance is 50%
    n_valzero = np.shape(np.where(Ytrain==0)[0])[0]
    n_valone  = np.shape(np.where(Ytrain==1)[0])[0]
    i_valzero = np.where(Ytrain==0)[0]
    i_valone  = np.where(Ytrain==1)[0]

    if n_valone > n_valzero:
        isubset_valone = np.random.choice(i_valone,size=n_valzero,replace=False)
        i_valnew = np.sort(np.append(i_valzero,isubset_valone))
        Y_train_1D = Ytrain.isel(time_all=i_valnew,drop=True)
        X_train_w_NANS  = Xtrain[i_valnew]#.stack(z=('lat','lon'))
    elif n_valone < n_valzero:
        isubset_valzero = np.random.choice(i_valzero,size=n_valone,replace=False)
        i_valnew = np.sort(np.append(isubset_valzero,i_valone))
        Y_train_1D = Ytrain.isel(time_all=i_valnew,drop=True)
        X_train_w_NANS  = Xtrain[i_valnew]#.stack(z=('lat','lon'))
    else:
        X_train_w_NANS = Xtrain#.stack(z=('lat','lon'))

    #Validation
    # make Yval have equal 0s and 1s so that random chance is 50%
    n_valzero = np.shape(np.where(Yval==0)[0])[0]
    n_valone  = np.shape(np.where(Yval==1)[0])[0]
    i_valzero = np.where(Yval==0)[0]
    i_valone  = np.where(Yval==1)[0]

    if n_valone > n_valzero:
        isubset_valone = np.random.choice(i_valone,size=n_valzero,replace=False)
        i_valnew = np.sort(np.append(i_valzero,isubset_valone))
        Y_val_1D = Yval.isel(time_all=i_valnew,drop=True)
        X_val_w_NANS  = Xval[i_valnew]#.stack(z=('lat','lon'))
    elif n_valone < n_valzero:
        isubset_valzero = np.random.choice(i_valzero,size=n_valone,replace=False)
        i_valnew = np.sort(np.append(isubset_valzero,i_valone))
        Y_val_1D = Yval.isel(time_all=i_valnew,drop=True)
        X_val_w_NANS  = Xval[i_valnew]#.stack(z=('lat','lon'))
    else:
        X_val_w_NANS = Xval#.stack(z=('lat','lon'))

    #Testing
    # make Ytest have equal 0s and 1s so that random chance is 50%
    n_valzero = np.shape(np.where(Ytest==0)[0])[0]
    n_valone  = np.shape(np.where(Ytest==1)[0])[0]
    i_valzero = np.where(Ytest==0)[0]
    i_valone  = np.where(Ytest==1)[0]

    if n_valone > n_valzero:
        isubset_valone = np.random.choice(i_valone,size=n_valzero,replace=False)
        i_valnew = np.sort(np.append(i_valzero,isubset_valone))
        Y_test_1D = Ytest.isel(time=i_valnew,drop=True)
        X_test_w_NANS  = Xtest[i_valnew]#.stack(z=('lat','lon'))
    elif n_valone < n_valzero:
        isubset_valzero = np.random.choice(i_valzero,size=n_valone,replace=False)
        i_valnew = np.sort(np.append(isubset_valzero,i_valone))
        Y_test_1D = Ytest.isel(time=i_valnew,drop=True)
        X_test_w_NANS  = Xtest[i_valnew]#.stack(z=('lat','lon'))
    else:
        X_test_w_NANS = Xtest#.stack(z=('lat','lon'))

    X_train_stack = X_train_w_NANS.stack(z=('lat','lon'))
    X_train = X_train_stack.dropna(dim="z", how="any")

    X_val_stack = X_val_w_NANS.stack(z=('lat','lon'))
    X_val = X_val_stack.dropna(dim="z", how="any")

    # X_test_stack = X_test_w_NANS.stack(z=('lat','lon'))
    # X_test = X_test_stack.dropna(dim="z", how="any")

    X_test_stack = Xtest.stack(z=('lat','lon'))
    X_test = X_test_stack.dropna(dim="z", how="all")

    enc = preprocessing.OneHotEncoder()
    Y_train      = enc.fit_transform(np.array(Y_train_1D).reshape(-1, 1)).toarray()
    Y_val  = enc.fit_transform(np.array(Y_val_1D).reshape(-1, 1)).toarray()
    #Y_test  = enc.fit_transform(np.array(Y_test_1D).reshape(-1, 1)).toarray()
    Y_test  = enc.fit_transform(np.array(Ytest).reshape(-1, 1)).toarray()

    del Y_val1_inputtime, Yval, Y_val1_inputtime1, Ytrain
    del X_val1_inputtime1, X_all1_inputtime
    del val_centered_weighted_rolling_avg, centered_weighted_rolling_avg

    # Evaluate Test Member and Metrics

    perc = 80 #(100-perc)% of predictions
    conf_level = 'most'
    subset = 'correct'
    X1= X_test.copy(deep=True)

    #>>>>>>>>Analyze test data on trained model<<<<<<<<
    NETWORK_SEED=0
    model, LOSS = make_model() 
    ind = 0

    stored_accuracy = np.zeros(len(RANDOM_SEED))
    for SEED in RANDOM_SEED:
        #if ind < num_experiments:
        model.load_weights(ddir_MODEL+PREDICTOR_VAR+'_'+str(lead_weeks)+'wklead_operationalseed_testens'+str(testing_ens)+'_seed'+str(SEED)+'.h5')
        results_test = model.evaluate(X_test,Y_test,verbose = 0) #prints out model evaluation 
        stored_accuracy[ind] = results_test[2]
        ind += 1

    seed_winner_trained = np.argmax(np.array(stored_accuracy[:]))

    for SEED in [RANDOM_SEED[seed_winner_trained]]:
        #**** To evaluate XAI on validation or testing, just change Ptest and Cttest_true)********

        Ptest = model.predict(X_test)    #predicted values on test data - softmax output of confidence 
        Cptest_pred = Ptest.argmax(axis=1)     #0,1 of predicted  
        Cttest_true = Y_test.argmax(axis=1) #true values on validation 

        #<<<<<<<<<<<Split most confident data into correct and incorrect>>>>>>>
        #softmax:

        max_logits = np.max(Ptest,axis=-1)
        if conf_level == 'most':
            i_cover = np.where(max_logits >= np.percentile(max_logits, perc))[0]

        if conf_level == 'least':
            i_cover = np.where(max_logits < np.percentile(max_logits, perc))[0]

        cat_true_conf = Cttest_true[i_cover]
        cat_pred_conf = Cptest_pred[i_cover]

        #if subset == 'correct':
        #location of correct predictions
        icorr = np.where(Cptest_pred[i_cover] - Cttest_true[i_cover] == 0)[0]
        X1_subset = X1[i_cover][icorr] #index Xtest with indicies of CONFIDENT & CORRECT predictions; for output compositing, replace X1 with Y variable 
        Y_true_subset = cat_true_conf[icorr]
        Y_pred_subset = cat_pred_conf[icorr]
        time = X_test.time[icorr]

        #if subset == 'incorrect': #confident but incorrect/wrong prediction
        #location of incorrect predictions
        i_incorr = np.where(Cptest_pred[i_cover] - Cttest_true[i_cover] != 0)[0]
        X1_subset_F = X1[i_cover][i_incorr] #index Xtest with indicies of INCORRECT predictions
        Y_false_subset = cat_true_conf[i_incorr]
        Y_pred_subset = cat_pred_conf[i_incorr]
        time_F = X_test.time[i_incorr]

        sample = X1_subset.copy()
        icat = np.where(Y_true_subset == 0)[0]
        time_class = time[icat]
        true_neg = X1_subset[icat]

        sample = X1_subset.copy()
        icat = np.where(Y_true_subset == 1)[0]
        time_class = time[icat]
        true_pos = X1_subset[icat]

        sample = X1_subset_F.copy()
        icat = np.where(Y_false_subset == 1)[0]  # 
        time_class = time_F[icat]
        false_neg = X1_subset_F[icat]

        sample = X1_subset_F.copy()
        icat = np.where(Y_false_subset == 0)[0]  # 
        time_class = time_F[icat]
        false_pos = X1_subset_F[icat]
        
        chance_hit = int((len(true_pos) + len(false_pos))*(len(true_pos) + len(false_neg))/len(cat_true_conf))
        
        
    threat_score_value = metrics.threat_score(len(true_pos), len(false_pos), len(false_neg))
    print("Threat Score:", threat_score_value)
    
    gilbert_score_value = metrics.gilbert_skill_score(len(true_pos), len(false_pos), len(false_neg), chance_hit)
    print("Gilbert Skill Score:", gilbert_score_value)

    threat_score_l8[exp_ind] = threat_score_value
    gilbert_skill_score_l8[exp_ind] = gilbert_score_value
    exp_ind += 1

/Users/marcodia/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 265)]             0         
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 dense (Dense)               (None, 160)               42560     
                                                                 
 dense_1 (Dense)             (None, 192)               30912     
                                                                 
 dense_2 (Dense)             (None, 2)                 386       
                                                                 
Total params: 73,858
Trainable params: 73,858
Non-trainable params: 0
_________________________________________________________________
385/385 [==============================] - 0s 1ms/step
Th

/Users/marcodia/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 265)]             0         
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 dense (Dense)               (None, 160)               42560     
                                                                 
 dense_1 (Dense)             (None, 192)               30912     
                                                                 
 dense_2 (Dense)             (None, 2)                 386       
                                                                 
Total params: 73,858
Trainable params: 73,858
Non-trainable params: 0
_________________________________________________________________
385/385 [==============================] - 0s 1ms/step
Th

/Users/marcodia/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 265)]             0         
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 dense (Dense)               (None, 160)               42560     
                                                                 
 dense_1 (Dense)             (None, 192)               30912     
                                                                 
 dense_2 (Dense)             (None, 2)                 386       
                                                                 
Total params: 73,858
Trainable params: 73,858
Non-trainable params: 0
_________________________________________________________________
385/385 [==============================] - 0s 994us/step


/Users/marcodia/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 265)]             0         
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 dense (Dense)               (None, 160)               42560     
                                                                 
 dense_1 (Dense)             (None, 192)               30912     
                                                                 
 dense_2 (Dense)             (None, 2)                 386       
                                                                 
Total params: 73,858
Trainable params: 73,858
Non-trainable params: 0
_________________________________________________________________
385/385 [==============================] - 0s 1ms/step
Th

/Users/marcodia/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 265)]             0         
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 dense (Dense)               (None, 160)               42560     
                                                                 
 dense_1 (Dense)             (None, 192)               30912     
                                                                 
 dense_2 (Dense)             (None, 2)                 386       
                                                                 
Total params: 73,858
Trainable params: 73,858
Non-trainable params: 0
_________________________________________________________________
385/385 [==============================] - 0s 986us/step


In [237]:
gilbert_skill_score_l8

array([ 0.02796421,  0.06510851, -0.00462963,  0.00575448,  0.04327301])

In [238]:
#threat_xr8 = xr.DataArray(threat_score_l8)
#threat_xr8.to_netcdf(ddir_saveinfo+'threat_score_l8_FOO.nc', mode='w')

gilbert_xr8 = xr.DataArray(gilbert_skill_score_l8)
gilbert_xr8.to_netcdf(ddir_saveinfo+'gilbert_skill_score_l8_FOO.nc', mode='w')


In [239]:
int(chance_hit)

359

In [240]:
cat_true_conf.shape

(2460,)

In [241]:
chance_hitX = (len(true_pos) + len(false_pos))*(len(true_pos) + len(false_neg))/(len(cat_true_conf)*len(cat_true_conf))

In [242]:
chance_hitX

0.14628908057373258

In [243]:
len(true_pos) + len(false_pos)+len(true_neg)+len(false_neg)

2460

In [244]:
true_pos.shape

(414, 265)

In [245]:
len(false_pos)

1007

In [246]:
len(false_neg)

209